In [1]:
import os
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model, layers, optimizers, losses, metrics, datasets

root_URL = 'C:\\Users\\lenovo\\Desktop\\'

In [2]:
# 加载数据
pickle_file = root_URL + 'svhn_dataset.pkl'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_images = save['train_images']
    train_labels = save['train_labels']
    test_images = save['test_images']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Train Shape:', train_images.shape, train_labels.shape)
    print('Test Shape:', test_images.shape, test_labels.shape)

Train Shape: (33401, 64, 64) (33401, 6)
Test Shape: (13068, 64, 64) (13068, 6)


In [3]:
# 预处理函数
def preprocess(x, y):
    x = tf.expand_dims(x, axis=-1) # 在axis=3扩展一维
    y = [y[:,0],y[:,1],y[:,2],y[:,3],y[:,4],y[:,5]]
    return x, y

In [4]:
train_images, train_labels = preprocess(train_images,train_labels)
test_images, test_labels = preprocess(test_images,test_labels)
print(train_images.shape,test_images.shape)

(33401, 64, 64, 1) (13068, 64, 64, 1)


In [5]:
# 创建网络模型并装配模型
inputs = Input(shape = (64,64,1))

model = layers.BatchNormalization()(inputs)
model = layers.Conv2D(64, (3,3), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(128, (3,3), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(256, (3,3), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.BatchNormalization()(model)
model = layers.Conv2D(512, (3,3), padding='same', activation='relu')(model)
model = layers.MaxPool2D(pool_size=2)(model)

model = layers.Dropout(0.5)(model)

model = layers.Flatten()(model)

model = layers.Dense(1024, activation='relu')(model)
model = layers.Dense(512, activation='relu')(model)

output_0 = layers.Dense(6, activation='softmax')(model)# length
output_1 = layers.Dense(11, activation='softmax')(model)# digits
output_2 = layers.Dense(11, activation='softmax')(model)
output_3 = layers.Dense(11, activation='softmax')(model)
output_4 = layers.Dense(11, activation='softmax')(model)
output_5 = layers.Dense(11, activation='softmax')(model)

outputs = [output_0, output_1, output_2, output_3, output_4, output_5]

In [5]:
# VGG16
inputs = Input(shape = (56,112,1))

#layer_1
model = layers.Conv2D(64,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(inputs)
model = layers.Conv2D(64,(3,3),strides=(1,1),padding='same',data_format='channels_last',kernel_initializer='uniform',activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_2
model = layers.Conv2D(128,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(model)
model = layers.Conv2D(128,(2,2),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_3
model = layers.Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(256, (1, 1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_4
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same', data_format='channels_last', activation='relu')(model)
model = layers.Conv2D(512, (1,1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

#layer_5
model = layers.Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu')(model)
model = layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same', data_format='channels_last', activation='relu')(model)
model = layers.Conv2D(512, (1,1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu')(model)
model = layers.MaxPooling2D((2,2))(model)

model = layers.Flatten()(model)  #拉平
model = layers.Dense(1024,activation='relu')(model)

output_0 = layers.Dense(6, activation='softmax')(model)# length
output_1 = layers.Dense(11, activation='softmax')(model)# digits
output_2 = layers.Dense(11, activation='softmax')(model)
output_3 = layers.Dense(11, activation='softmax')(model)
output_4 = layers.Dense(11, activation='softmax')(model)
output_5 = layers.Dense(11, activation='softmax')(model)

outputs = [output_0, output_1, output_2, output_3, output_4, output_5]

In [6]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   640         batch_normalization[0][0]        
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 32, 64)   0           conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
# 设置回调功能
filepath = 'my_model.h5' # 保存模型地址
saved_model = tf.keras.callbacks.ModelCheckpoint(filepath, verbose = 1) # 回调保存模型功能
tensorboard = tf.keras.callbacks.TensorBoard() # 回调可视化数据功能

In [8]:
# 执行训练与验证
history = model.fit(train_images, train_labels, 
                    batch_size = 64, epochs = 100,
                    validation_split = 0.2,
                    callbacks = [saved_model, tensorboard])

Train on 26720 samples, validate on 6681 samples
Epoch 1/100
26688/26720 [============================>.] - ETA: 0s - loss: 6.0385 - dense_2_loss: 0.9828 - dense_3_loss: 1.7257 - dense_4_loss: 1.9775 - dense_5_loss: 1.0653 - dense_6_loss: 0.2762 - dense_7_loss: 0.0111 - dense_2_accuracy: 0.7874 - dense_3_accuracy: 0.5433 - dense_4_accuracy: 0.4568 - dense_5_accuracy: 0.7484 - dense_6_accuracy: 0.9558 - dense_7_accuracy: 0.9980
Epoch 00001: saving model to my_model.h5
26720/26720 [==============================] - 55s 2ms/sample - loss: 6.0337 - dense_2_loss: 0.9808 - dense_3_loss: 1.7229 - dense_4_loss: 1.9748 - dense_5_loss: 1.0633 - dense_6_loss: 0.2759 - dense_7_loss: 0.0111 - dense_2_accuracy: 0.7875 - dense_3_accuracy: 0.5436 - dense_4_accuracy: 0.4571 - dense_5_accuracy: 0.7485 - dense_6_accuracy: 0.9558 - dense_7_accuracy: 0.9980 - val_loss: 5.4980 - val_dense_2_loss: 0.9833 - val_dense_3_loss: 1.5098 - val_dense_4_loss: 1.9787 - val_dense_5_loss: 0.7983 - val_dense_6_loss: 0.21

In [9]:
model = tf.keras.models.load_model('my_model.h5')
model.evaluate(test_images, test_labels)

13068/13068 [==============================] - 9s 698us/sample - loss: 3.4352 - dense_2_loss: 0.6260 - dense_3_loss: 0.8858 - dense_4_loss: 1.2030 - dense_5_loss: 0.5900 - dense_6_loss: 0.1071 - dense_7_loss: 0.0197 - dense_2_accuracy: 0.9646 - dense_3_accuracy: 0.9172 - dense_4_accuracy: 0.9027 - dense_5_accuracy: 0.9597 - dense_6_accuracy: 0.9940 - dense_7_accuracy: 0.9998s - loss: 3.6456 - dense_2_loss: 0.6871 - dense_3_loss: 0.9154 - dense_4_loss: 1.2803 - dense_5_loss: 0.6100 - dense_6_loss: 0.1232 - dense_7_loss: 0.0297 - dense_2_accuracy: 0.9645 - dense_3_accuracy: 0.9156 - dense_4_accuracy: 0.9022 - dense_5_accuracy: 0.9596 - dens


[3.4351802192808547,
 0.62604886,
 0.88582575,
 1.2030374,
 0.59001064,
 0.107068464,
 0.01966282,
 0.9645699,
 0.91720235,
 0.9027395,
 0.9596725,
 0.99403125,
 0.99984694]

In [5]:
model = tf.keras.models.load_model('my_model.h5')
correct = 0
test_len = len(test_images)
for i in range(test_len):
    x = tf.expand_dims(test_images[i], axis=0)
    y1 = tf.expand_dims(test_labels[0][i], axis=0)
    y2 = tf.expand_dims(test_labels[1][i], axis=0)
    y3 = tf.expand_dims(test_labels[2][i], axis=0)
    y4 = tf.expand_dims(test_labels[3][i], axis=0)
    y5 = tf.expand_dims(test_labels[4][i], axis=0)
    y6 = tf.expand_dims(test_labels[5][i], axis=0)
    y = [y1,y2,y3,y4,y5,y6]
    eval = model.evaluate(x, y,verbose=0)
    acc = tf.reduce_prod(eval[8:])
    
    if acc == 1:
        correct += 1

accuracy = correct / test_len
print(accuracy)

0.830119375573921
